In [20]:
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [21]:
import os
from tqdm import notebook, tnrange, tqdm
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

img_height=393
img_width=397

In [22]:
id_long=next(os.walk("/home/shelia/AllFiles/Research_tuly/pyDeepInsight/pyDeepInsight_SRT/examples/data/desmeth/long"))[2]
print("no of images : ", len(id_long))

id_short=next(os.walk("/home/shelia/AllFiles/Research_tuly/pyDeepInsight/pyDeepInsight_SRT/examples/data/desmeth/short"))[2]
print("no of images : ", len(id_short))

no of images :  111
no of images :  48


In [ ]:
X1 = np.zeros((len(id_long), img_height, img_width, 1), dtype=np.float32)
X2 = np.zeros((len(id_short), img_height, img_width, 1), dtype=np.float32)

In [23]:
from torchvision import datasets, transforms
from skimage.io import imread, imshow, concatenate_images
#import helper

In [24]:
data_dir1 = "/home/shelia/AllFiles/Research_tuly/pyDeepInsight/pyDeepInsight_SRT/examples/data/desmeth"
#data_dir2=  "/home/shelia/AllFiles/Research_tuly/pyDeepInsight/pyDeepInsight_SRT/examples/data/desmeth/short"
transform = transforms.Compose([transforms.Resize(224),
transforms.ToTensor()])
im_dataset1 = datasets.ImageFolder(data_dir1, transform=transform)
#im_dataset2 = datasets.ImageFolder(data_dir2, transform=transform)

In [ ]:
im_dataset1

In [25]:
train_size = int(0.8 * len(im_dataset1))
test_size = len(im_dataset1) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(im_dataset1, [train_size, test_size])
dataloaders={'train':torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True),'test':torch.utils.data.DataLoader(test_dataset,batch_size=10, shuffle=True)}

In [41]:
im_dataset1.shape

AttributeError: 'ImageFolder' object has no attribute 'shape'

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [27]:
import timm
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader

In [61]:
net = timm.create_model('resnet50', pretrained=True, num_classes=2).to(device)
net = net.to(device)
net.fc=torch.nn.Sequential(
               torch.nn.Linear(2048, 128),
               torch.nn.ReLU(inplace=True),
               torch.nn.Linear(128, 2)).to(device)


In [62]:
batch_size=10
preprocess = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=net.default_cfg['mean'], std=net.default_cfg['std']),
])


trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [63]:
criterion = nn.CrossEntropyLoss()
#ptimizer = optim.SGD(net.parameters(),lr=1e-02,momentum=2,)
optimizer = optim.Adam(net.parameters(),lr=0.0001)

In [64]:
from IPython.display import clear_output,display

In [65]:
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':

                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                epoch_loss = running_loss / len(train_dataset)
                epoch_acc = running_corrects.double() / len(train_dataset)
            else:
                epoch_loss = running_loss / len(test_dataset)
                epoch_acc = running_corrects.double() / len(test_dataset)


            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [68]:
model_trained = train_model(net, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train loss: 0.0847, acc: 0.9764
test loss: 0.9199, acc: 0.6562
Epoch 2/10
----------
train loss: 0.0486, acc: 0.9843
test loss: 0.7412, acc: 0.6250
Epoch 3/10
----------
train loss: 0.0701, acc: 0.9764
test loss: 0.8224, acc: 0.8125
Epoch 4/10
----------
train loss: 0.0380, acc: 0.9921
test loss: 0.9504, acc: 0.7500
Epoch 5/10
----------
train loss: 0.0182, acc: 1.0000
test loss: 1.0139, acc: 0.6875
Epoch 6/10
----------
train loss: 0.0169, acc: 0.9921
test loss: 0.8658, acc: 0.6875
Epoch 7/10
----------
train loss: 0.0106, acc: 1.0000
test loss: 0.9735, acc: 0.6875
Epoch 8/10
----------
train loss: 0.0510, acc: 0.9921
test loss: 1.0842, acc: 0.6562
Epoch 9/10
----------
train loss: 0.0172, acc: 1.0000
test loss: 0.9246, acc: 0.6250
Epoch 10/10
----------
train loss: 0.0130, acc: 1.0000
test loss: 0.9105, acc: 0.7188
